Julia does not have automatic conversion. Mathematical operators are just functions with special syntax, and the arguments of functions are never automatically converted. Applying mathematical operations to a wide variety of mixed argument types is just an extreme case of polymorphic multiple dispatch. Julia comes with pre-defined catch-all dispatch rules for mathematical operators, invoked when no specific implementation exists for some combination of operand types.These catch-all rules first promote all operands to a common type using user-definable promotion rules, and then invoke a specialized implementation of the operator in question for the resulting values, now of the same type. User-defined types can easily participate in this promotion system by defining methods for conversion to and from other types, and providing a handful of promotion rules defining what types they should promote to when mixed with other types.

# Conversion
For conversion to a specific type one needs to use the convert function. Let us say we want to convert a variable of type Float64 to Int64.

In [1]:
x = 10.0
typeof(x)

Float64

In [2]:
y = convert(Int64, x)
typeof(y)

Int64

Let us look at how one can convert an array to a different type.
    

In [3]:
A = Any[1 2 3; 4 5 6]

2×3 Matrix{Any}:
 1  2  3
 4  5  6

In [4]:
B = convert(Array{Float64}, A)

2×3 Matrix{Float64}:
 1.0  2.0  3.0
 4.0  5.0  6.0

## Conversion of userdefined types
Consider a struct MyComplex

In [5]:
struct MyComplex <: Number
    x::Float64   # Real part of number
    y::Float64   # Imaginary part of number
end

In [6]:
Base.convert(::Type{MyComplex}, x::Float64) = MyComplex(x, 0)

In [7]:
convert(MyComplex, 5)

LoadError: MethodError: no method matching MyComplex(::Int64)
[0mClosest candidates are:
[0m  (::Type{T})(::T) where T<:Number at boot.jl:772
[0m  MyComplex(::Any, [91m::Any[39m) at In[5]:2
[0m  (::Type{T})([91m::AbstractChar[39m) where T<:Union{AbstractChar, Number} at char.jl:50
[0m  ...

The type of the first argument of this method is Type{MyComplex}, the only instance of which is MyComplex. Thus, this method is only invoked when the first argument is the type value MyComplex. Notice the syntax used for the first argument: the argument name is omitted prior to the :: symbol, and only the type is given. This is the syntax in Julia for a function argument whose type is specified but whose value does not need to be referenced by name.

# Promotion
Promotion refers to converting values of mixed types to a single common type. Although it is not strictly necessary, it is generally implied that the common type to which the values are converted can faithfully represent all of the original values. In this sense, the term "promotion" is appropriate since the values are converted to a "greater" type – i.e. one which can represent all of the input values in a single common type.
Promotion to a common "greater" type is performed in Julia by the promote function, which takes any number of arguments, and returns a tuple of the same number of values, converted to a common type, or throws an exception if promotion is not possible. The most common use case for promotion is to convert numeric arguments to a common type:

In [8]:
promote(1, 2.5)

(1.0, 2.5)

In [9]:
promote(1, 2.5, 3, 3//4)

(1.0, 2.5, 3.0, 0.75)

So what is the use of promote? We have defined our own complex type above. Let us try to add a variable of type Float64 to it. Mathematically such an operation makes sense.

In [10]:
c = MyComplex(5, 5)
function add(c1::MyComplex, c2::MyComplex)
    return MyComplex(c1.x+c2.x, c1.y+c2.y)
end
    
y = add(c, 5.0)

LoadError: MethodError: no method matching add(::MyComplex, ::Float64)
[0mClosest candidates are:
[0m  add(::MyComplex, [91m::MyComplex[39m) at In[10]:2

We have not defined addition between  the types MyComplex and Float64. We can define such a method. But then we would have to define all the three combinations
(MyComplex,Float64) ...
Instead we define a promote_rule

In [11]:
Base.promote_rule(::Type{Float64}, ::Type{MyComplex}) = MyComplex 

In [15]:
add(x, y) = add(promote(x, y)...)

add (generic function with 2 methods)

In [16]:
add(c, 5.0)

MyComplex(10.0, 5.0)